# Deploy the finetuned stanford_alpaca model on Amazon SageMaker

As we have finetuned the model, next we will show you how to deploy the model on SageMaker.

In this notebook, we explore how to host a large language model on SageMaker using the [Large Model Inference](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-inference.html) container that is optimized for hosting large models using DJLServing. DJLServing is a high-performance universal model serving solution powered by the Deep Java Library (DJL) that is programming language agnostic. To learn more about DJL and DJLServing, you can refer to our recent [blog post](https://aws.amazon.com/blogs/machine-learning/deploy-large-models-on-amazon-sagemaker-using-djlserving-and-deepspeed-model-parallel-inference/).

## Create a SageMaker Model for Deployment
As a first step, we'll import the relevant libraries and configure several global variables such as the hosting image that will be used nd the S3 location of our model artifacts

In [1]:
import sagemaker
from sagemaker.model import Model
from sagemaker import serializers, deserializers
from sagemaker import image_uris
import boto3
import os
import time
import json
import jinja2
from pathlib import Path

In [2]:
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment

s3_client = boto3.client("s3") # client to intreract with S3 API
sm_client = boto3.client("sagemaker")  # client to intreract with SageMaker
smr_client = boto3.client("sagemaker-runtime") # client to intreract with SageMaker Endpoints
jinja_env = jinja2.Environment() # jinja environment to generate model configuration templates

In [3]:
# lookup the inference image uri based on our current region
djl_inference_image_uri = (
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117"
)

In [5]:
pretrained_model_location = "s3://sagemaker-us-east-1-xxxxxxx/llama/output/2023-07-02-09-00-13/llama_out/"# Change to the model artifact path in S3 which we get from the fine tune job
print(f"Pretrained model will be downloaded from ---- > {pretrained_model_location}")

Pretrained model will be downloaded from ---- > s3://sagemaker-us-east-1-633205212955/llama/output/2023-07-02-09-00-13/llama_out/


In [19]:
inference_image_uri = '763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117'


## Deploying a Large Language Model using Hugging Face Accelerate
The DJL Inference Image which we will be utilizing ships with a number of built-in inference handlers for a wide variety of tasks including:
- `text-generation`
- `question-answering`
- `text-classification`
- `token-classification`

You can refer to this [GitRepo](https://github.com/deepjavalibrary/djl-serving/tree/master/engines/python/setup/djl_python) for a list of additional handlers and available NLP Tasks. <br>
These handlers can be utilized as is without having to write any custom inference code. We simply need to create a `serving.properties` text file with our desired hosting options and package it up into a `tar.gz` artifact.

Lets take a look at the `serving.properties` file that we'll be using for our first example

In [7]:
!mkdir accelerate_src

In [8]:
%%writefile accelerate_src/serving.template
engine=Python
option.entryPoint=djl_python.huggingface
option.s3url={{ s3url }}
option.task=text-generation
option.device_map=auto
option.load_in_8bit=TRUE

Writing accelerate_src/serving.template


In [9]:
# we plug in the appropriate model location into our `serving.properties` file based on the region in which this notebook is running
template = jinja_env.from_string(Path("accelerate_src/serving.template").open().read())
Path("accelerate_src/serving.properties").open("w").write(template.render(s3url=pretrained_model_location))
!pygmentize accelerate_src/serving.properties | cat -n

     1	engine=Python
     2	option.entryPoint=djl_python.huggingface
     3	option.s3url=s3://sagemaker-us-east-1-633205212955/llama/output/2023-07-02-09-00-13/llama_out/
     4	option.task=text-generation
     5	option.device_map=auto
     6	option.load_in_8bit=TRUE


There are a few options specified here. Lets go through them in turn<br>
1. `engine` - specifies the engine that will be used for this workload. In this case we'll be hosting a model using the [DJL Python Engine](https://github.com/deepjavalibrary/djl-serving/tree/master/engines/python)
2. `option.entryPoint` - specifies the entrypoint code that will be used to host the model. djl_python.huggingface refers to the `huggingface.py` module from [djl_python repo](https://github.com/deepjavalibrary/djl-serving/tree/master/engines/python/setup/djl_python).  
3. `option.s3url` - specifies the location of the model files. Alternativelly an `option.model_id` option can be used instead to specifiy a model from Hugging Face Hub (e.g. `EleutherAI/gpt-j-6B`) and the model will be automatically downloaded from the Hub. The s3url approach is recommended as it allows you to host the model artifact within your own environment and enables faster deployments by utilizing optimized approach within the DJL inference container to transfer the model from S3 into the hosting instance 
4. `option.task` - This is specific to the `huggingface.py` inference handler and specifies for which task this model will be used
5. `option.device_map` - Enables layer-wise model partitioning through [Hugging Face Accelerate](https://huggingface.co/docs/accelerate/usage_guides/big_modeling#designing-a-device-map). With `option.device_map=auto`, Accelerate will determine where to put each **layer** to maximize the use of your fastest devices (GPUs) and offload the rest on the CPU, or even the hard drive if you don’t have enough GPU RAM (or CPU RAM). Even if the model is split across several devices, it will run as you would normally expect.
6. `option.load_in_8bit` - Quantizes the model weights to int8 thereby greatly reducing the memory footprint of the model from the initial FP32. See this [blog post](https://huggingface.co/blog/hf-bitsandbytes-integration) from Hugging Face for additional information 

For more information on the available options, please refer to the [SageMaker Large Model Inference Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-configuration.html)

Our initial approach here is to utilize the built-in functionality within Hugging Face Transformers to enable Large Language Model hosting. 

We place the `serving.properties` file into a tarball and upload it to S3

In [14]:
!tar czvf acc_model.tar.gz accelerate_src/ 

accelerate_src/
accelerate_src/serving.template
accelerate_src/serving.properties
accelerate_src/.ipynb_checkpoints/
accelerate_src/.ipynb_checkpoints/serving-checkpoint.properties
accelerate_src/.ipynb_checkpoints/serving-checkpoint.template
accelerate_src/.ipynb_checkpoints/requirements-checkpoint.txt
accelerate_src/requirements.txt


In [15]:
s3_code_prefix = "llama/deploy/code"

code_artifact = sess.upload_data("acc_model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-east-1-633205212955/llama/deploy/code/acc_model.tar.gz


## Deploy Model to a SageMaker Endpoint
With a helper function we can now deploy our endpoint and invoke it with some sample inputs

In [20]:
def deploy_model(image_uri, model_data, role, endpoint_name, instance_type, sagemaker_session):
    
    """Helper function to create the SageMaker Endpoint resources and return a predictor"""
    model = Model(
            image_uri=image_uri, 
              model_data=model_data, 
              role=role
             )
    
    model.deploy(
        initial_instance_count=1,
        instance_type=instance_type,
        endpoint_name=endpoint_name
        )
    
    # our requests and responses will be in json format so we specify the serializer and the deserializer
    predictor = sagemaker.Predictor(
        endpoint_name=endpoint_name, 
        sagemaker_session=sagemaker_session, 
        serializer=serializers.JSONSerializer(), 
        deserializer=deserializers.JSONDeserializer())
    
    return predictor

In [21]:
# creates a unique endpoint name
endpoint_name = sagemaker.utils.name_from_base("alpaca-7B")
print(f"Our endpoint will be called {endpoint_name}")

Our endpoint will be called alpaca-7B-2023-07-02-09-14-48-435


In [22]:
# deployment will take about 10 minutes
predictor = deploy_model(image_uri=inference_image_uri, 
                            model_data=code_artifact, 
                            role=role, 
                            endpoint_name=endpoint_name, 
                            instance_type="ml.g4dn.4xlarge", 
                            sagemaker_session=sess)

------------!

Let's run an example with a basic text generation prompt Large model inference is

In [23]:
predictor.predict({ 
                    "inputs" : "Large model inference is", 
                    "parameters": { "max_length": 50, "temperature": 0.5 }
                })

[{'generated_text': 'Large model inference is a challenging task, and the performance of the model is often limited by the computational cost of the inference. In this paper, we propose a novel approach to accelerate the inference of large models. The key idea is to'}]

In [24]:
print(predictor.predict({ "inputs":
                                """Message: Support has been terrible for 2 weeks...
                                Sentiment: Negative
                                ###
                                Message: I love your API, it is simple and so fast!
                                Sentiment: Positive
                                ###
                                Message: GPT-J has been released 12 months ago.
                                Sentiment: Neutral
                                ###
                                Message: The responsiveness of your team has been amazing, thank you so much!
                                Sentiment:""",
                      "parameters": { "max_length": 50, "temperature": 0.5 }
                     }
                    )[0]["generated_text"])

Message: Support has been terrible for 2 weeks...
                                Sentiment: Negative
                                ###
                                Message: I love your API, it is simple and so fast!
                                Sentiment: Positive
                                ###
                                Message: GPT-J has been released 12 months ago.
                                Sentiment: Neutral
                                ###
                                Message: The responsiveness of your team has been amazing, thank you so much!
                                Sentiment: Pos


Finally Let's do a quick benchmark to see what kind of latency we can expect from this model

In [25]:
%%timeit -n3 -r1
predictor.predict({ 
                    "inputs" : "Large model inference is", 
                    "parameters": { "max_length": 50, "temperature": 0.5 }
                })

8.43 s ± 0 ns per loop (mean ± std. dev. of 1 run, 3 loops each)


In [ ]:
# Clean up the endpoint before proceeding
predictor.delete_endpoint()

## Reference

[sagemaker-hosting/Large-Language-Model-Hosting/](https://github.com/aws-samples/sagemaker-hosting/tree/main/Large-Language-Model-Hosting)